In [ ]:
#|default_exp migrate

# migrate
> Utilities for migrating to nbdev

In [ ]:
#|export
from nbdev.process import first_code_ln
from nbdev.read import *
from nbdev.processors import yml2dict, filter_fm
from nbdev.read import read_nb, config_key
from nbdev.sync import write_nb
from nbdev.clean import process_write
from nbdev.showdoc import show_doc
from fastcore.all import *
import shutil

## Aliases For Redirects

fastpages and Jekyll style blog posts automatically generate URLs based on the category and filename.  Furhtermore, we want to create [Quarto aliases](https://quarto.org/docs/websites/website-navigation.html#redirects) to new Quarto generated paths to minimize broken links when you migrate from older versions of nbdev or fastpages.

In [ ]:
#|export
def _cat_slug(d):
    "Get the partial slug from the category front matter."
    slug = '/'.join(sorted(d.get('categories', '')))
    return '/' + slug if slug else ''

In [ ]:
#|hide
_fm1 = NB('../tests/2020-09-01-fastcore.ipynb')
test_eq(_cat_slug(_fm1.fmdict), '/fastai/fastcore')

_fm2 = NB('../tests/2020-02-20-test.ipynb')
test_eq(_cat_slug(_fm2.fmdict), '/jupyter')

In [ ]:
#|export
def _file_slug(fname): 
    "Get the partial slug from the filename."
    p = Path(fname)
    dt = '/'+p.name[:10].replace('-', '/')+'/'
    return dt + p.stem[11:]    

In [ ]:
#|hide
test_eq(_file_slug('../tests/2020-09-01-fastcore.ipynb'), '/2020/09/01/fastcore')

In [ ]:
#|export
_re_dt = re.compile(r'^\d{4}-\d{2}-\d{2}')

def _alias(fm:dict, p:Path):
    p = Path(p)
    if not _re_dt.search(p.name): return {}
    return {'aliases': [f"{fm.pop('permalink').strip()}"] if 'permalink' in fm else [f'{_cat_slug(fm) + _file_slug(p)}']}

In [ ]:
#|export
def nb_alias_fm(path):
    "Fix slugs for fastpages and jekyll compatibility."
    nb = NB(path)
    nb.update_raw_fm(_alias(nb.fmdict, path)) #use the combined markdown & raw front matter to determine the alias
    return nb

In [ ]:
# This notebook already has existing raw front matter
_raw_fm_cell = nb_alias_fm('../tests/2020-09-01-fastcore.ipynb').cells[0].source
assert 'aliases:\n- /fastcore/' in _raw_fm_cell
print(_raw_fm_cell)

In [ ]:
# This notebook doesn't have any existing raw front matter
_raw_fm_cell = nb_alias_fm('../tests/2020-02-20-test.ipynb').cells[0].source
assert 'aliases:\n- /jupyter/2020/02/20/test' in _raw_fm_cell
print(_raw_fm_cell)

## Callouts

In fastpages, there was a markdown shortuct for callouts for `Note`, `Tip`, `Important` and `Warning` with block quotes.  Since Quarto has its own [callout blocks](https://quarto.org/docs/authoring/callouts.html#callout-types) with markdown syntax, we do not implement these shortcuts in nbdev.  Instead, we offer a manual conversion utility for these callouts so that you can migrate from fastpages to Quarto.

In [ ]:
#|export
_re_callout = re.compile(r'^>\s(Warning|Note|Important|Tip):(.*)', flags=re.MULTILINE)
def _co(x): return "\n:::{.callout-"+x[1].lower()+"}\n\n" + f"{x[2].strip()}\n\n" + ":::\n"
def convert_callout(s): 
    "Convert nbdev v1 to v2 callouts."
    return _re_callout.sub(_co, s)

For example, the below markdown:

In [ ]:
_callouts="""
## Boxes / Callouts

> Warning: There will be no second warning!

Other text

> Important: Pay attention! It's important.

> Tip: This is my tip.

> Note: Take note of `this.`
"""

Gets converted to:

In [ ]:
#| echo:false
_c = convert_callout(_callouts)
assert '> Tip:' not in _c
assert 'Other text' in _c
print(_c)

In [ ]:
#|export
def _listify(s): return s.splitlines() if type(s) == str else s

def _nb_repl_callouts(nb):
    "Replace nbdev v1 with v2 callouts."
    for cell in nb['cells']:
        if cell.get('source') and cell.get('cell_type') == 'markdown':
            cell['source'] = ''.join([convert_callout(c) for c in _listify(cell['source'])])
    return nb

## Convert Notebook Directives

nbdev v2 directives start with a `#|` whereas v1 directives were comments without a pipe `|`.  Furthermore, there are some directives with different names that need to be changed for v2.

In [ ]:
#|export
_dirmap = merge({k:'code-fold: true' for k in ['collapse', 'collapse_input', 'collapse_hide']}, {'collapse_show':'code-fold: show'})
def _subv1(s): return _dirmap.get(s, s)

In [ ]:
#|export
def _re_v1():
    d = ['default_exp', 'export', 'exports', 'exporti', 'hide', 'hide_input', 'collapse_show', 'collapse',
         'collapse_hide', 'collapse_input', 'hide_output',  'default_cls_lvl']
    d += L(config_key('tst_flags', path=False)).filter()
    d += [s.replace('_', '-') for s in d] # allow for hyphenated version of old directives
    _tmp = '|'.join(list(set(d)))
    return re.compile(f"^[ \f\v\t]*?(#)\s*({_tmp})(?!\S)", re.MULTILINE)

def _repl_directives(code_str): 
    def _fmt(x): return f"#| {_subv1(x[2].replace('-', '_').strip())}"
    return _re_v1().sub(_fmt, code_str)

for example, if any of the lines below are valid nbdev v1 directives, they replaced with a `#|`, and their names are aliased to new directive names where appropriate:

In [ ]:
#|hide
_test_dir = """
#default_exp
 #export
# collapse-show
#collapse-hide
#collapse
# collapse_output
not_dir='#export'
# hide_input
foo
"""
test_eq(_repl_directives(_test_dir),
"""
#| default_exp
#| export
#| code-fold: show
#| code-fold: true
#| code-fold: true
# collapse_output
not_dir='#export'
#| hide_input
foo
""")

In [ ]:
#|export

def repl_v1dir(nb):
    "Replace nbdev v1 with v2 directives."
    for cell in nb['cells']:
        if cell.get('source') and cell.get('cell_type') == 'code':
            ss = listify(cell['source'])
            first_code = first_code_ln(ss, re_pattern=_re_v1())
            if not first_code: first_code = len(ss)
            if not ss: pass
            else: cell['source'] = ''.join([_repl_directives(c) for c in ss[:first_code]] + ss[first_code:])
    return nb

In [ ]:
#|hide
_code = _test_dir.splitlines(True)

tst = {'cell_type': 'code', 'execution_count': 26,
       'metadata': {'hide_input': True, 'meta': 23},
       'outputs': [{'execution_count': 2,
                    'data': {
                        'application/vnd.google.colaboratory.intrinsic+json': {'type': 'string'},
                        'plain/text': ['sample output',]
                    }, 'output': 'super'}],
       'source': _code}
nb = {'metadata': {'kernelspec': 'some_spec', 'jekyll': 'some_meta', 'meta': 37}, 'cells': [tst]}

This is a cell with relevant values before we replace directives:

In [ ]:
nb['cells'][0]['source']

And after:

In [ ]:
_nb = repl_v1dir(nb)['cells'][0]['source']
test_eq(_nb.strip(),
"""
#| default_exp
#| export
#| code-fold: show
#| code-fold: true
#| code-fold: true
# collapse_output
not_dir='#export'
# hide_input
foo
""".strip())

### Migrate notebooks

In [ ]:
#|export
def migrate_nb(path, overwrite=False):
    "Migrate nbdev v1 and fastpages notebooks to nbdev v2."
    nb = compose(nb_alias_fm, _nb_repl_callouts, repl_v1dir)(path)
    if overwrite: write_nb(nb, path)
    return nb

`migrate_nb` will do the following:
    
- Place aliases in raw front matter so that your old URLs can redirect to Quarto URLs
- Convert directives from v1 to v2 
- Convert callouts from v1 to v2

Here is an example (also note how existing front matter that is already there, such as `execute: echo: false` is retained):

In [ ]:
_nb = migrate_nb('../tests/2022-08-10-migrate.ipynb')
assert '\n:::{.callout-note}\n\nthis is a note\n\n:::\n' in _nb.text
assert "#| hide\nprint('hello')" in _nb.text
assert '- /bar/foo/2022/08/10/migrate' in _nb.text[0]
_nb.print_txt()

:::{.callout-note}

`migrate_nb` will not move markdown front matter to raw front matter.  `nbdev` will make a best effort to infer front matter from Markdown, however, it is recommended you use raw cells and front matter for customizing the display of your notebook per the [Quarto docs](https://quarto.org/docs/reference/formats/html.html).

:::

### Migrate Markdown Files

In [ ]:
#|export
_re_fm_md = re.compile(r'^---(.*\S+.)?---', flags=re.DOTALL)

def _md_fmdict(txt):
    "Get front matter as a dict from a markdown file."
    m = _re_fm_md.match(txt)
    return yml2dict(m.group(1)) if m else {}

In [ ]:
#|hide
_mdtxt = Path('../tests/2020-01-14-test-markdown-post.md').read_text()
test_eq(_md_fmdict(_mdtxt), 
                {'toc': True,
               'layout': 'post',
               'description': 'A minimal example of using markdown with fastpages.',
               'categories': ['markdown'],
               'title': 'An Example Markdown Post'})

In [ ]:
#|export
def migrate_md(path, overwrite=False):
    "Make fastpages front matter in markdown files quarto compliant."
    p = Path(path)
    md = p.read_text()
    fm = _md_fmdict(md)
    if fm:
        fm = filter_fm(merge(_alias(fm, p), fm))
        txt = _re_fm_md.sub(dict2fm(fm), md)
        if overwrite: p.write_text(txt)
        return txt
    else: return md 

Here is what the front matter of a markdown post looks like before:

In [ ]:
#|eval: false
print(run('head -n13 ../tests/2020-01-14-test-markdown-post.md'))

And this is what the front matter looks like after:

In [ ]:
#|hide
_res = migrate_md('../tests/2020-01-14-test-markdown-post.md')
assert """---
aliases:
- /markdown/2020/01/14/test-markdown-post
categories:
- markdown
description: A minimal example of using markdown with fastpages.
title: An Example Markdown Post
toc: true

---""" in _res

In [ ]:
print(_res[:310])

In [ ]:
#|export
@call_parse
def nbdev_migrate(
    path:str = '.', # A path to search
    file_glob:str = '*.ipynb', # A file glob
    no_skip:bool=False, # Do not skip directories beginning with an underscore
):
    "Convert all directives and callouts in `fname` from v1 to v2"
    _skip_re = None if no_skip else '^[_.]'
    if path is None: path = config_key("nbs_path")
    if Path(path).is_file(): file_glob=None
    for f in globtastic(path, file_glob=file_glob, skip_folder_re=_skip_re): 
        if f.suffix == '.ipynb': migrate_nb(f, overwrite=True)
        if f.suffix == '.md': migrate_md(f, overwrite=True)

In [ ]:
#|hide
def _nb2str(p): return '\n'.join(NB(p).text)
try:
    _orig =  Path('../tests/2020-02-20-test.ipynb') # nbdev v1 notebook
    _tmp =  Path('../tests/2020-02-20-test-COPY.ipynb') # A copy of this nb that will be migrated
    shutil.copy(_orig, _tmp)
    nbdev_migrate(_tmp)

    assert ':::{.callout-warning}' not in _nb2str(_orig) and ':::{.callout-warning}' in _nb2str(_tmp)
    assert '#| code-fold: true' not in _nb2str(_orig) and '#| code-fold: true' in _nb2str(_tmp)
    assert '#| single-value' not in _nb2str(_tmp)

finally:
    if _tmp.exists(): _tmp.unlink() # missing_ok not in python 3.7

In [ ]:
#|hide
try:
    _orig =  Path('../tests/2020-01-14-test-markdown-post.md') 
    _tmp =  Path('../tests/2020-01-14-test-markdown-post-COPY.md')
    shutil.copy(_orig, _tmp)
    nbdev_migrate(_tmp)
    
    assert 'aliases:' in _tmp.read_text() and 'aliases:' not in _orig.read_text()
finally:
    if _tmp.exists(): _tmp.unlink() # missing_ok not in python 3.7

## Export -

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()